In [3]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.wait import WebDriverWait 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time 
import pandas as pd
import pymongo
from pymongo import MongoClient
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Document, Text, Integer
from elasticsearch.helpers import bulk
import numpy as np
import re

In [4]:
df = pd.read_csv('dataframe_finale.csv',delimiter=';')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4200 entries, 0 to 4199
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Title            4200 non-null   object
 1   Artist           4200 non-null   object
 2   Rank             4200 non-null   int64 
 3   Last Week        4200 non-null   object
 4   Peak Positon     4200 non-null   int64 
 5   Weeks on charts  4200 non-null   int64 
 6   Date             4200 non-null   object
 7   Genre            4200 non-null   object
 8   Producers        4200 non-null   object
 9   Writers          4179 non-null   object
 10  Distributor      4193 non-null   object
dtypes: int64(3), object(8)
memory usage: 361.1+ KB


In [5]:
df['Last Week'] = df['Last Week'].replace("-",0)
df['Last Week']= df['Last Week'].astype('int64')
df['Date']=pd.to_datetime(df['Date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4200 entries, 0 to 4199
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Title            4200 non-null   object        
 1   Artist           4200 non-null   object        
 2   Rank             4200 non-null   int64         
 3   Last Week        4200 non-null   int64         
 4   Peak Positon     4200 non-null   int64         
 5   Weeks on charts  4200 non-null   int64         
 6   Date             4200 non-null   datetime64[ns]
 7   Genre            4200 non-null   object        
 8   Producers        4200 non-null   object        
 9   Writers          4179 non-null   object        
 10  Distributor      4193 non-null   object        
dtypes: datetime64[ns](1), int64(4), object(6)
memory usage: 361.1+ KB


In [6]:
data = df.to_dict('records')
data

[{'Title': 'Shut Down',
  'Artist': 'BLACKPINK',
  'Rank': 1,
  'Last Week': 0,
  'Peak Positon': 1,
  'Weeks on charts': 1,
  'Date': Timestamp('2022-10-01 00:00:00'),
  'Genre': 'Pop, K-Pop (케이팝), Modern Classical, Orchestral, Girl Group, South Korea, Genius Korea',
  'Producers': 'Teddy Park & 24',
  'Writers': 'Teddy Park, Danny Chung & Vince (KOR)',
  'Distributor': 'None'},
 {'Title': 'Pink Venom',
  'Artist': 'BLACKPINK',
  'Rank': 2,
  'Last Week': 4,
  'Peak Positon': 1,
  'Weeks on charts': 5,
  'Date': Timestamp('2022-10-01 00:00:00'),
  'Genre': 'Pop, Korean, Hip-Hop, K-Pop (케이팝), Girl Group, South Korea, Genius Korea',
  'Producers': 'Teddy Park, 24, R.Tee & IDO (KOR)',
  'Writers': 'Teddy Park, , R.Tee, Danny Chung & IDO (KOR)',
  'Distributor': 'None'},
 {'Title': "I'm Good (Blue)",
  'Artist': 'David Guetta & Bebe Rexha',
  'Rank': 3,
  'Last Week': 2,
  'Peak Positon': 2,
  'Weeks on charts': 4,
  'Date': Timestamp('2022-10-01 00:00:00'),
  'Genre': 'Pop, Rap, Hip-Hop,

In [7]:
for i,song in enumerate(data):
    song['Genre'] = re.split(', | & ', song['Genre'])
    song['Producers'] = re.split(', | & ', song['Producers'])
    try:
        song['Writers'] = re.split(', | & ', song['Writers'])
    except TypeError:
        data[i]["Writers"] = "None"
        song['Writers'] = re.split(', | & ', song['Writers'])
    try:
        song['Distributor'] = re.split(', | & ', song['Distributor'])
    except TypeError:
        data[i]["Distributor"] = "None"
        song['Distributor'] = re.split(', | & ', song['Distributor'])
data

[{'Title': 'Shut Down',
  'Artist': 'BLACKPINK',
  'Rank': 1,
  'Last Week': 0,
  'Peak Positon': 1,
  'Weeks on charts': 1,
  'Date': Timestamp('2022-10-01 00:00:00'),
  'Genre': ['Pop',
   'K-Pop (케이팝)',
   'Modern Classical',
   'Orchestral',
   'Girl Group',
   'South Korea',
   'Genius Korea'],
  'Producers': ['Teddy Park', '24'],
  'Writers': ['Teddy Park', 'Danny Chung', 'Vince (KOR)'],
  'Distributor': ['None']},
 {'Title': 'Pink Venom',
  'Artist': 'BLACKPINK',
  'Rank': 2,
  'Last Week': 4,
  'Peak Positon': 1,
  'Weeks on charts': 5,
  'Date': Timestamp('2022-10-01 00:00:00'),
  'Genre': ['Pop',
   'Korean',
   'Hip-Hop',
   'K-Pop (케이팝)',
   'Girl Group',
   'South Korea',
   'Genius Korea'],
  'Producers': ['Teddy Park', '24', 'R.Tee', 'IDO (KOR)'],
  'Writers': ['Teddy Park', '', 'R.Tee', 'Danny Chung', 'IDO (KOR)'],
  'Distributor': ['None']},
 {'Title': "I'm Good (Blue)",
  'Artist': 'David Guetta & Bebe Rexha',
  'Rank': 3,
  'Last Week': 2,
  'Peak Positon': 2,
  'Wee

In [8]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200,"scheme": "http"}])

In [9]:
es.ping()

True

In [10]:
def generate_data(data):
    for docu in data:
        yield {
            "_index": "billboard",
            "_type": "song",
            "_source": {k:v if v else None for k,v in docu.items()},
        }
        
if es.indices.exists('billboard')==True:
    es.indices.delete(index='billboard')
    bulk(es, generate_data(data))
else :
    bulk(es, generate_data(data))

C:\Users\camil\AppData\Local\Temp\ipykernel_19312\2795846709.py:9: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  if es.indices.exists('billboard')==True:
c:\Users\camil\AppData\Local\Programs\Python\Python310\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchWarning)


In [11]:
es.indices.exists('billboard')

C:\Users\camil\AppData\Local\Temp\ipykernel_19312\1081182445.py:1: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.exists('billboard')


True

In [12]:
result = es.search(index="billboard", query= {"match_all": {}})
result

{'took': 1,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 3000, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'billboard',
    '_type': 'song',
    '_id': 'hkQWiYYBm5-drd1mF53s',
    '_score': 1.0,
    '_source': {'Title': 'Shut Down',
     'Artist': 'BLACKPINK',
     'Rank': 1,
     'Last Week': None,
     'Peak Positon': 1,
     'Weeks on charts': 1,
     'Date': '2022-10-01T00:00:00',
     'Genre': ['Pop',
      'K-Pop (케이팝)',
      'Modern Classical',
      'Orchestral',
      'Girl Group',
      'South Korea',
      'Genius Korea'],
     'Producers': ['Teddy Park', '24'],
     'Writers': ['Teddy Park', 'Danny Chung', 'Vince (KOR)'],
     'Distributor': ['None']}},
   {'_index': 'billboard',
    '_type': 'song',
    '_id': 'h0QWiYYBm5-drd1mF53s',
    '_score': 1.0,
    '_source': {'Title': 'Pink Venom',
     'Artist': 'BLACKPINK',
     'Rank': 2,
     'Last Week': 4,
     'Peak Positon': 1,
   

In [14]:
N_DOCS = result['hits']['total']['value']
infos = f"{N_DOCS} document{'s' if N_DOCS> 1 else '' } correspondent à la requêtes qui a pris {result['took']} ms"
infos

'3000 documents correspondent à la requêtes qui a pris 1 ms'

In [14]:
!curl http://localhost:9200/_cat/indices?v #!curl http://elasticsearch:9200/_cat/indices?v

health status index                    uuid                   pri rep docs.count docs.deleted store.size pri.store.size
green  open   .kibana_task_manager_1   EXIwN7TRSVqNquk2ayLb4A   1   0          2            1     16.8kb         16.8kb
green  open   .apm-agent-configuration UL5dxL77QXCnt2-BKIT2aQ   1   0          0            0       283b           283b
yellow open   billboard                Te7KZU7WRnmZNaaUhkQWGA   1   1       4200            0      1.6mb          1.6mb
green  open   .kibana_1                jodusnl1QCuev0azrBTcxg   1   0         25           10     63.8kb         63.8kb


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   600  100   600    0     0   4621      0 --:--:-- --:--:-- --:--:--  4687
curl: (3) URL using bad/illegal format or missing URL
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0curl: (6) Could not resolve host: elasticsearch


In [15]:
# cherche toutes les chansons avec pour artiste BLACKPINK
QUERY = {
    "query": {
        "term": {
            "Artist.keyword": "BLACKPINK"
        }
    }
}
result = es.search(index="billboard", body=QUERY)
N_DOCS = result['hits']['total']['value']
f"{N_DOCS} document{'s' if N_DOCS> 1 else '' } correspondent à la requêtes qui a pris {result['took']} ms"

C:\Users\camil\AppData\Local\Temp\ipykernel_21236\753701021.py:9: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  result = es.search(index="billboard", body=QUERY)


'51 documents correspondent à la requêtes qui a pris 5 ms'

In [16]:
[elt['_source'] for elt in result["hits"]["hits"]]

[{'Title': 'Shut Down',
  'Artist': 'BLACKPINK',
  'Rank': 1,
  'Last Week': None,
  'Peak Positon': 1,
  'Weeks on charts': 1,
  'Date': '2022-10-01T00:00:00',
  'Genre': ['Pop',
   'K-Pop (케이팝)',
   'Modern Classical',
   'Orchestral',
   'Girl Group',
   'South Korea',
   'Genius Korea'],
  'Producers': ['Teddy Park', '24'],
  'Writers': ['Teddy Park', 'Danny Chung', 'Vince (KOR)'],
  'Distributor': ['None']},
 {'Title': 'Pink Venom',
  'Artist': 'BLACKPINK',
  'Rank': 2,
  'Last Week': 4,
  'Peak Positon': 1,
  'Weeks on charts': 5,
  'Date': '2022-10-01T00:00:00',
  'Genre': ['Pop',
   'Korean',
   'Hip-Hop',
   'K-Pop (케이팝)',
   'Girl Group',
   'South Korea',
   'Genius Korea'],
  'Producers': ['Teddy Park', '24', 'R.Tee', 'IDO (KOR)'],
  'Writers': ['Teddy Park', '', 'R.Tee', 'Danny Chung', 'IDO (KOR)'],
  'Distributor': ['None']},
 {'Title': 'Typa Girl',
  'Artist': 'BLACKPINK',
  'Rank': 16,
  'Last Week': None,
  'Peak Positon': 16,
  'Weeks on charts': 1,
  'Date': '2022-10

In [17]:
# affiche le titre et le rang pour le resultat de la requete
{elt['_source']['Title']:elt['_source']['Rank'] for elt in result["hits"]["hits"]}

{'Shut Down': 4,
 'Pink Venom': 10,
 'Typa Girl': 16,
 'Hard To Love': 27,
 'The Happiest Girl': 34,
 'Yeah Yeah Yeah': 43,
 'Tally': 52,
 'Ready For Love': 60}

In [18]:
# result['hits']['hits']
QUERY = {
    "query": {
        "term": {
            "Title.keyword": "Christmas"
        }
    }
}
result = es.search(index="billboard", body=QUERY)
N_DOCS = result['hits']['total']['value']
f"{N_DOCS} document{'s' if N_DOCS> 1 else '' } correspondent à la requêtes qui a pris {result['took']} ms"

C:\Users\camil\AppData\Local\Temp\ipykernel_21236\3674638867.py:9: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  result = es.search(index="billboard", body=QUERY)


'0 document correspondent à la requêtes qui a pris 1 ms'

In [17]:
# affiche les top 10 d'artistes qui ont le plus de chansons dans le billboard
QUERY ={
  "aggs": {
    "artist": {
      "terms": {
        "field": "Artist.keyword",
      }
    }
  }
}
result = es.search(index="billboard", body=QUERY)
# result["aggregations"]["artist"]["buckets"]
result["aggregations"]

C:\Users\camil\AppData\Local\Temp\ipykernel_19312\1721589055.py:11: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  result = es.search(index="billboard", body=QUERY)


{'artist': {'doc_count_error_upper_bound': 0,
  'sum_other_doc_count': 3396,
  'buckets': [{'key': 'Taylor Swift', 'doc_count': 150},
   {'key': 'Bad Bunny', 'doc_count': 102},
   {'key': 'Ed Sheeran', 'doc_count': 95},
   {'key': 'SZA', 'doc_count': 89},
   {'key': 'The Weeknd', 'doc_count': 79},
   {'key': 'Eminem', 'doc_count': 72},
   {'key': 'Harry Styles', 'doc_count': 59},
   {'key': 'Arctic Monkeys', 'doc_count': 56},
   {'key': 'BLACKPINK', 'doc_count': 51},
   {'key': 'Drake & 21 Savage', 'doc_count': 51}]}}

In [56]:
# requete correspondant au graphe évolution du rang d'une chanson (As it Was) au cours des semaines
QUERY = {
  "aggs": {
    "2": {
      "date_histogram": {
        "field": "Date",
        "calendar_interval": "1w",
        "time_zone": "Europe/Paris",
        "min_doc_count": 1
      },
      "aggs": {
        "1": {
          "avg": {
            "field": "Rank"
          }
        }
      }
    }
  },
  "size": 0,
  "_source": {
    "excludes": []
  },
  "stored_fields": [
    "*"
  ],
  "script_fields": {},
  "docvalue_fields": [
    {
      "field": "Date",
      "format": "date_time"
    }
  ],
  "query": {
    "bool": {
      "must": [],
      "filter": [
        {
          "bool": {
            "should": [
              {
                "match_phrase": {
                  "Title.keyword": "As It Was"
                }
              }
            ],
            "minimum_should_match": 1
          }
        }
      ],
      "should": [],
      "must_not": []
    }
  }
}
result = es.search(index="billboard", body=QUERY)

x_values, y_values = [],[]
for i in range(len(result["aggregations"]["2"]["buckets"])):
  aggregations_x = result["aggregations"]["2"]["buckets"][i]['key_as_string']
  aggregations_y = result["aggregations"]["2"]["buckets"][i]['1']['value']
  x_values.append(aggregations_x)
  y_values.append(aggregations_y)

import plotly.express as px

fig = px.line(x=x_values,y=y_values,# title=f"Evolution du rang {query} de au cours des semaines")
  title="Evolution du rang de chanson au cours des semaines")
fig.update_layout(
    xaxis_title="Dates by Weeks",
    yaxis_title="Rank"
)
fig.update_yaxes(autorange="reversed")
fig_json = fig.to_json()
import json,plotly
# fig_json = json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)
fig_json

C:\Users\camil\AppData\Local\Temp\ipykernel_19312\4008524846.py:56: DeprecationWarning:

The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information



'{"data": [{"hovertemplate": "x=%{x}<br>y=%{y}<extra></extra>", "legendgroup": "", "line": {"color": "#636efa", "dash": "solid"}, "marker": {"symbol": "circle"}, "mode": "lines", "name": "", "orientation": "v", "showlegend": false, "x": ["2022-09-26T00:00:00.000+02:00", "2022-10-03T00:00:00.000+02:00", "2022-10-10T00:00:00.000+02:00", "2022-10-17T00:00:00.000+02:00", "2022-10-24T00:00:00.000+02:00", "2022-10-31T00:00:00.000+01:00", "2022-11-07T00:00:00.000+01:00", "2022-11-14T00:00:00.000+01:00", "2022-11-21T00:00:00.000+01:00", "2022-11-28T00:00:00.000+01:00", "2022-12-05T00:00:00.000+01:00", "2022-12-12T00:00:00.000+01:00", "2022-12-19T00:00:00.000+01:00", "2022-12-26T00:00:00.000+01:00", "2023-01-02T00:00:00.000+01:00", "2023-01-09T00:00:00.000+01:00", "2023-01-16T00:00:00.000+01:00", "2023-01-23T00:00:00.000+01:00", "2023-01-30T00:00:00.000+01:00", "2023-02-06T00:00:00.000+01:00"], "xaxis": "x", "y": [4.0, 3.0, 3.0, 3.0, 4.0, 17.0, 14.0, 16.0, 6.0, 7.0, 9.0, 11.0, 20.0, 24.5, 6.0, 

In [35]:
# requete correspondant au graphe évolution d'un genre (Christmas) au cours des semaines
QUERY = {
  "aggs": {
    "2": {
      "date_histogram": {
        "field": "Date",
        "calendar_interval": "1w",
        "time_zone": "Europe/Paris",
        "min_doc_count": 1
      }
    }
  },
  "size": 0,
  "_source": {
    "excludes": []
  },
  "stored_fields": [
    "*"
  ],
  "script_fields": {},
  "docvalue_fields": [
    {
      "field": "Date",
      "format": "date_time"
    }
  ],
  "query": {
    "bool": {
      "must": [],
      "filter": [
        {
          "bool": {
            "should": [
              {
                "match_phrase": {
                  "Genre.keyword": "Christmas"
                }
              }
            ],
            "minimum_should_match": 1
          }
        }
      ],
      "should": [],
      "must_not": []
    }
  }
}

In [ ]:
# requete qui retourne le classement des genres
QUERY = {
  "aggs": {
    "2": {
      "terms": {
        "field": "Genre.keyword",
        "order": {
          "_count": "desc"
        },
        "size": 60
      }
    }
  },
  "size": 0,
  "_source": {
    "excludes": []
  },
  "stored_fields": [
    "*"
  ],
  "script_fields": {},
  "docvalue_fields": [
    {
      "field": "Date",
      "format": "date_time"
    }
  ],
  "query": {
    "bool": {
      "must": [],
      "filter": [
        {
          "match_all": {}
        }
      ],
      "should": [],
      "must_not": []
    }
  }
}

In [21]:
# retourne tout les titres genre kpop
QUERY ={
  "query": {
    "bool": {
      "must": [],
      "filter": [
        {
          "bool": {
            "should": [
              {
                "match_phrase": {
                  "Genre.keyword": "K-Pop (케이팝)"
                }
              }
            ],
            "minimum_should_match": 1
          }
        }
      ],
      "should": [],
      "must_not": []
    }
  }
}
result = es.search(index="billboard", body=QUERY)
[elt['_source'] for elt in result["hits"]["hits"]]
N_DOCS = result['hits']['total']['value']
f"{N_DOCS} document{'s' if N_DOCS> 1 else '' } correspondent à la requêtes qui a pris {result['took']} ms"

C:\Users\camil\AppData\Local\Temp\ipykernel_21236\3423818596.py:25: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  result = es.search(index="billboard", body=QUERY)


'175 documents correspondent à la requêtes qui a pris 12 ms'

In [22]:
# retourne genre avec Pop ET Rap
QUERY={
    "query": {
    "bool": {
      "must": [],
      "filter": [
        {
          "bool": {
            "filter": [
              {
                "bool": {
                  "should": [
                    {
                      "match_phrase": {
                        "Genre.keyword": "Pop"
                      }
                    }
                  ],
                  "minimum_should_match": 1
                }
              },
              {
                "bool": {
                  "should": [
                    {
                      "match_phrase": {
                        "Genre.keyword": "Rap"
                      }
                    }
                  ],
                  "minimum_should_match": 1
                }
              }
            ]
          }
        }
      ],
      "should": [],
      "must_not": []
    }
  }
}
result = es.search(index="billboard", body=QUERY)
[elt['_source'] for elt in result["hits"]["hits"]]
N_DOCS = result['hits']['total']['value']
f"{N_DOCS} document{'s' if N_DOCS> 1 else '' } correspondent à la requêtes qui a pris {result['took']} ms"

C:\Users\camil\AppData\Local\Temp\ipykernel_21236\1010801425.py:43: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  result = es.search(index="billboard", body=QUERY)


'304 documents correspondent à la requêtes qui a pris 12 ms'

In [23]:
# chaque mois top des chansons qui apparaissent le plus sur le billbaord (pas en fonction de leur rang)
QUERY = {
  "aggs": {
    "song_by_month": {
      "date_histogram": {
        "field": "Date",
        "calendar_interval": "month"
      },
      "aggs": {
        "title": {
          "terms": {
            "field": "Title.keyword"
          }
        }
      }
    }
  }
}
result = es.search(index="billboard", body=QUERY)
result["aggregations"]["song_by_month"]["buckets"]

C:\Users\camil\AppData\Local\Temp\ipykernel_21236\3956439160.py:19: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  result = es.search(index="billboard", body=QUERY)


[{'key_as_string': '2022-10-01T00:00:00.000Z',
  'key': 1664582400000,
  'doc_count': 1000,
  'title': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 950,
   'buckets': [{'key': '2 Be Loved (Am I Ready)', 'doc_count': 5},
    {'key': '505', 'doc_count': 5},
    {'key': 'About Damn Time', 'doc_count': 5},
    {'key': 'After Like', 'doc_count': 5},
    {'key': "All Too Well (Taylor's Version)", 'doc_count': 5},
    {'key': 'Another Love', 'doc_count': 5},
    {'key': 'As It Was', 'doc_count': 5},
    {'key': 'Atlantis', 'doc_count': 5},
    {'key': 'B.O.T.A. (Baddest Of Them All)', 'doc_count': 5},
    {'key': 'Baby Shark', 'doc_count': 5}]}},
 {'key_as_string': '2022-11-01T00:00:00.000Z',
  'key': 1667260800000,
  'doc_count': 800,
  'title': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 755,
   'buckets': [{'key': 'Miss You', 'doc_count': 7},
    {'key': 'Die For You', 'doc_count': 6},
    {'key': '505', 'doc_count': 4},
    {'key': 'About Damn Time', 'doc_

In [24]:
QUERY= {
  "aggs": {
    "avg_weeks_on_chart": {
      "avg": {
        "field": "Weeks on charts"
      }
    }
  }
}
result = es.search(index="billboard", body=QUERY)
result["aggregations"]["avg_weeks_on_chart"]

C:\Users\camil\AppData\Local\Temp\ipykernel_21236\1238742633.py:10: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  result = es.search(index="billboard", body=QUERY)


{'value': 31.349047619047617}

In [25]:
QUERY = {
  "aggs": {
    "top_song_per_month": {
      "date_histogram": {
        "field": "Date",
        "calendar_interval": "month"
      }
    }
  }
}
result = es.search(index="billboard", body=QUERY)
result["aggregations"]["top_song_per_month"]

C:\Users\camil\AppData\Local\Temp\ipykernel_21236\2806668549.py:11: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  result = es.search(index="billboard", body=QUERY)


{'buckets': [{'key_as_string': '2022-10-01T00:00:00.000Z',
   'key': 1664582400000,
   'doc_count': 1000},
  {'key_as_string': '2022-11-01T00:00:00.000Z',
   'key': 1667260800000,
   'doc_count': 800},
  {'key_as_string': '2022-12-01T00:00:00.000Z',
   'key': 1669852800000,
   'doc_count': 1000},
  {'key_as_string': '2023-01-01T00:00:00.000Z',
   'key': 1672531200000,
   'doc_count': 1000},
  {'key_as_string': '2023-02-01T00:00:00.000Z',
   'key': 1675209600000,
   'doc_count': 400}]}

In [26]:
# QUERY ={
#   "aggs": {
#     "title": {
#       "terms": {
#         "field": "Title.keyword"
#       },
#       "aggs": {
#         "artists": {
#           "terms": {
#             "field": "Artist.keyword"
#           }
#         }
#       }
#     }
#   }
# }
QUERY={
  "aggs": {
    "title": {
      "terms": {
        "field": "Title.keyword"
      },
      "aggs": {
        "artists": {
          "terms": {
            "field": "Artist.keyword"
          },
          "aggs": {
            "unique_artists": {
              "cardinality": {
                "field": "Artist.keyword"
              }
            }
          }
        }
      }
    }
  }
}

result = es.search(index="billboard", body=QUERY)
result = result["aggregations"]["title"]["buckets"]
result

C:\Users\camil\AppData\Local\Temp\ipykernel_21236\2297239962.py:41: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  result = es.search(index="billboard", body=QUERY)


[{'key': 'Die For You',
  'doc_count': 27,
  'artists': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 'The Weeknd',
     'doc_count': 21,
     'unique_artists': {'value': 1}},
    {'key': 'Joji', 'doc_count': 6, 'unique_artists': {'value': 1}}]}},
 {'key': 'Miss You',
  'doc_count': 22,
  'artists': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 'Oliver Tree & Robin Schulz',
     'doc_count': 18,
     'unique_artists': {'value': 1}},
    {'key': 'southstar', 'doc_count': 4, 'unique_artists': {'value': 1}}]}},
 {'key': 'About Damn Time',
  'doc_count': 21,
  'artists': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 'Lizzo',
     'doc_count': 21,
     'unique_artists': {'value': 1}}]}},
 {'key': 'Another Love',
  'doc_count': 21,
  'artists': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 'Tom Odell',
     'doc_count': 21,
     '

In [27]:
for i in range(len(result)):
  title = result[i]['key'],result[i]['doc_count']
  print(title)
for i in range(len(result)):
  artistesdict = result[i]['artists']['buckets']
  for j in range(len(artistesdict)):
    artiste = artistesdict[j]['key'],artistesdict[j]['doc_count']
    print(artiste)

('Die For You', 27)
('Miss You', 22)
('About Damn Time', 21)
('Another Love', 21)
('As It Was', 21)
('Baby Shark', 21)
('Bad Habit', 21)
('Bad Habits', 21)
('Believer', 21)
('Blinding Lights', 21)
('The Weeknd', 21)
('Joji', 6)
('Oliver Tree & Robin Schulz', 18)
('southstar', 4)
('Lizzo', 21)
('Tom Odell', 21)
('Harry Styles', 21)
('Pinkfong', 21)
('Steve Lacy', 21)
('Ed Sheeran', 21)
('Imagine Dragons', 21)
('The Weeknd', 21)


In [28]:
df

,Title,Artist,Rank,Last Week,Peak Positon,Weeks on charts,Date,Genre,Producers,Writers,Distributor
0,Shut Down,BLACKPINK,1,0,1,1,2022-10-01,"Pop, K-Pop (케이팝), Modern Classical, Orchestral...",Teddy Park & 24,"Teddy Park, Danny Chung & Vince (KOR)",None
1,Pink Venom,BLACKPINK,2,4,1,5,2022-10-01,"Pop, Korean, Hip-Hop, K-Pop (케이팝), Girl Group,...","Teddy Park, 24, R.Tee & IDO (KOR)","Teddy Park, , R.Tee, Danny Chung & IDO (KOR)",None
2,I'm Good (Blue),David Guetta & Bebe Rexha,3,2,2,4,2022-10-01,"Pop, Rap, Hip-Hop, Cloud Rap, Trap, Conscious ...",T-Minus & Nikhil Seetharam,"Kendrick Lamar, T-Minus & Nikhil Seetharam",Universal Music Group
3,As It Was,Harry Styles,4,1,1,25,2022-10-01,"Rock, Pop, Electronic, UK Pop, Dance Rock, Fun...",Tyler Johnson & Kid Harpoon,"Kid Harpoon, Tyler Johnson & Harry Styles",Sony Music Entertainment
4,Me Porto Bonito,Bad Bunny & Chencho Corleone,5,3,2,20,2022-10-01,"Pop, Reggaetón, Puerto Rico, En Español, Latin...","Subelo NEO, Lennex, MAG & La Paciencia",Chencho Corleone & Bad Bunny,The Orchard
...,...,...,...,...,...,...,...,...,...,...,...
4195,In The End,Linkin Park,196,193,163,12,2023-02-12,"Rap, Rock, Hard Rock, Alternative Metal, Rap M...",Don Gilmore,"Mr. Hahn, Rob Bourdon, Mike Shinoda, Chester B...",None
4196,Ferrari,James Hype & Miggy Dela Rosa,197,174,25,31,2023-02-12,"Pop, Deep House, House, Alternative Dance, Ele...",James Hype,"James Hype, Miggy Dela Rosa, Josh Grimmett, Jo...",None
4197,Just The Way You Are,Bruno Mars,198,0,162,12,2023-02-12,"R&B, Pop, Alternative R&B, Piano, Ballad, Adul...",Needlz & The Smeezingtons,"Philip Lawrence, Ari Levine, Needlz, Saint Cas...",None
4198,Call Out My Name,The Weeknd,199,181,97,31,2023-02-12,"Pop, R&B, Singer-Songwriter, Downtempo, Canada...",Ging,"Ging, Nicolas Jaar & The Weeknd",None
